## Performance of chronos on aquifer data

### Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import joblib

import time
from datasetsforecast.m3 import M3
from utilsforecast.losses import *
from utilsforecast.evaluation import evaluate
import torch
from chronos import ChronosPipeline

from sklearn.metrics import r2_score

### Chronos

#### Data preparation

In [ ]:
# Read the dataset
aquifer_by_stations = joblib.load('aquifer_by_stations.joblib')

In [ ]:
# Ensure the datetime
for key, data in aquifer_by_stations.items():
    data['date'] = pd.to_datetime(data['date'])

Try for only one station

In [ ]:
aquifer = aquifer_by_stations[85012]

Mean scaling

In [ ]:
def mean_scaling(x):
    mean = np.mean(np.abs(x))

    return x/mean

def standard_scaling(x):
    mean = np.mean(np.abs(x))
    s = np.std(x)

    return (x - mean)/s

def standard_unscaling(original, scaled):
    mean = np.mean(np.abs(original))
    s = np.std(original)

    return (scaled * s) + mean

In [ ]:
y = aquifer['altitude'].values
y_scaled = mean_scaling(y)

fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2)

ax1.plot(aquifer['date'], y, color='blue', label='Original')
ax1.set_ylabel('Daily visits')
ax1.legend()

ax2.plot(aquifer['date'], y_scaled, color='orange', label='Scaled')
ax2.set_ylabel('Daily visits (scaled)')
ax2.legend()

fig.autofmt_xdate()
plt.tight_layout()

#### Forecast

In [ ]:
# Installs
#%pip install -U git+https://github.com/amazon-science/chronos-forecasting.git
#%pip install neuralforecast

Single forecast

In [ ]:
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-large",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

horizon = 10
history = 100


chronos_tiny_preds = []

start = time.time()

y = aquifer['altitude'].values
y_scaled = standard_scaling(y)
y = torch.tensor(y_scaled[-history:-horizon])

forecast = pipeline.predict(
    context= y,
    prediction_length=horizon,
    num_samples=20
)

low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)


chronos_tiny_duration = time.time() - start

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(aquifer['date'][-100:], y_scaled[-100:], color="royalblue", label="historical data")
plt.plot(aquifer['date'][-horizon:], median, color="tomato", label="median forecast")
plt.fill_between(aquifer['date'][-horizon:], low, high, color="tomato", alpha=0.3, label="80% prediction interval")
plt.legend()
plt.grid()
plt.show()

Sinusoid data

In [ ]:
num_points = 1000

# 0 to 20pi range with 1000 points
time_a = np.linspace(0 , 20 * np.pi, num_points)
frequency = 1
amplitude = 0.01  # Amplitude of the sine wave

# Generate the sine wave data
sinusoid = amplitude * np.sin(frequency * time_a)

# Shift the curve up by 1
shifted_sinusoid = sinusoid + 1

In [ ]:
# Plot the shifted sinusoidal curve
plt.plot(shifted_sinusoid, label='Shifted Sinusoid')
plt.title('Shifted Sinusoidal Curve Around 1')
plt.xlabel('Time')
plt.ylabel('Value')
plt.axhline(y=1, color='r', linestyle='--', label='y=1')
plt.legend()
plt.show()

In [ ]:
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-tiny",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

horizon = 50
history = 1000


chronos_tiny_preds = []


y = torch.tensor(shifted_sinusoid[:-horizon])

forecast = pipeline.predict(
    context= y,
    prediction_length=horizon,
    num_samples=20
)

low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)

In [ ]:
forecast_index = range(num_points - horizon, num_points)

plt.figure(figsize=(8, 4))
plt.plot(shifted_sinusoid, color="royalblue", label="historical data")
plt.plot(forecast_index, median, color="tomato", label="median forecast")
plt.fill_between(forecast_index, low, high, color="tomato", alpha=0.3, label="80% prediction interval")
plt.legend()
plt.grid()
plt.show()

Sinusoid data with scaling

In [ ]:
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-tiny",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

horizon = 50
history = 1000


chronos_tiny_preds = []

shifted_sinusoid_scaled = standard_scaling(shifted_sinusoid)
y = torch.tensor(shifted_sinusoid_scaled[:-horizon])

forecast = pipeline.predict(
    context= y,
    prediction_length=horizon,
    num_samples=20
)

low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)

# Unscale the forescasts
low = standard_unscaling(shifted_sinusoid, low)
median = standard_unscaling(shifted_sinusoid, median)
high = standard_unscaling(shifted_sinusoid, high)

In [ ]:
forecast_index = range(num_points - horizon, num_points)

plt.figure(figsize=(8, 4))
plt.plot(shifted_sinusoid, color="royalblue", label="historical data")
plt.plot(forecast_index, median, color="tomato", label="median forecast")
plt.fill_between(forecast_index, low, high, color="tomato", alpha=0.3, label="80% prediction interval")
plt.legend()
plt.grid()
plt.show()

<span style="color:red"><sup>!!! From the sinusoid experiment we found out, that if the data is pretty constant (small deviations), we should use standard scaling on the data, before passing it to chronos. Otherwise most of the predictions can fall into the same bit.</sub></span>

Sinusoid data with scaling (longer prediction period)

In [ ]:
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-tiny",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

# Parameters
day_len = 100
horizon = 5

# List for storing the r2 scores
r2_scores = [[] for _ in range(5)]


# List for storing the predictions
predictions = [[] for _ in range(5)]

shifted_sinusoid_scaled = standard_scaling(shifted_sinusoid)
y = torch.tensor(shifted_sinusoid_scaled[:-horizon])

# Iterate from day_len days before the end, to the last day
for i in range(day_len + (horizon-1), 0, -1):

    y = torch.tensor(shifted_sinusoid_scaled[:-i])

    

    forecast = pipeline.predict(
        context= y,
        prediction_length=horizon,
        num_samples=20
    )

    low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)
    median = standard_unscaling(shifted_sinusoid, median)

    # Store the results for every prediction horizon separately
    for i in range(5):
        predictions[i].append(median[i])

# Clean up the results
predictions[0] = predictions[0][-day_len:]
predictions[1] = predictions[1][3:-1]
predictions[2] = predictions[2][2:-2]
predictions[3] = predictions[3][1:-3]
predictions[4] = predictions[4][0:-4]

# Calculate the r2 scores and store them in a list
for i in range(5):
    r2_scores[i].append(r2_score(shifted_sinusoid[-day_len:], predictions[i]))

In [ ]:
forecast_index = range(0, 50)

plt.figure(figsize=(8, 4))
plt.plot(forecast_index, shifted_sinusoid[-50:], color="royalblue", label="historical data")
plt.plot(forecast_index, predictions[0][-50:], color="orange", label="median forecast")
plt.plot(forecast_index, predictions[4][-50:], color="tomato", label="median forecast")
plt.legend()
plt.grid()
plt.show()

Test dataset (from chronos github repository)

In [ ]:
import pandas as pd  # requires: pip install pandas
import torch
from chronos import ChronosPipeline

pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",
    device_map="cuda",  # use "cpu" for CPU inference and "mps" for Apple Silicon
    torch_dtype=torch.bfloat16,
)

df = pd.read_csv("https://raw.githubusercontent.com/AileenNielsen/TimeSeriesAnalysisWithPython/master/data/AirPassengers.csv")

# context must be either a 1D tensor, a list of 1D tensors,
# or a left-padded 2D tensor with batch as the first dimension
# forecast shape: [num_series, num_samples, prediction_length]
forecast = pipeline.predict(
    context=torch.tensor(df["#Passengers"]),
    prediction_length=12,
    num_samples=20,
)

In [ ]:
import matplotlib.pyplot as plt  # requires: pip install matplotlib
import numpy as np

forecast_index = range(len(df), len(df) + 12)
low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)

plt.figure(figsize=(8, 4))
plt.plot(df["#Passengers"], color="royalblue", label="historical data")
plt.plot(forecast_index, median, color="tomato", label="median forecast")
plt.fill_between(forecast_index, low, high, color="tomato", alpha=0.3, label="80% prediction interval")
plt.legend()
plt.grid()
plt.show()

Testing on aquifer data with relative differences in altitude

In [ ]:
relative_aquifer = aquifer_by_stations[85012]

In [ ]:
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-large",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

horizon = 3
history = 100


chronos_tiny_preds = []

start = time.time()

y = relative_aquifer['altitude_diff'].values
y_scaled = standard_scaling(y)
#y_scaled = y
y = torch.tensor(y_scaled[-history:-horizon])

forecast = pipeline.predict(
    context= y,
    prediction_length=horizon,
    num_samples=20
)

low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)


chronos_tiny_duration = time.time() - start

In [ ]:

plt.figure(figsize=(8, 4))
plt.plot(relative_aquifer['date'][-100:], y_scaled[-100:], color="royalblue", label="historical data")
plt.plot(relative_aquifer['date'][-horizon:], median, color="tomato", label="median forecast")
plt.fill_between(relative_aquifer['date'][-horizon:], low, high, color="tomato", alpha=0.3, label="80% prediction interval")
plt.legend()
plt.grid()
plt.show()

Testing multiple prediction horizons (without scaling)

In [ ]:
relative_aquifer = aquifer_by_stations[85065]

In [ ]:
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-large",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

horizon = 5
history = 100

day_1 = []
day_2 = []
day_3 = []
day_4 = []
day_5 = []

day_len = 200

# Iterate from day_len days before the end, to the last day
for i in range(day_len + 4, 0, -1):
    y = relative_aquifer['altitude_diff'].values
    y = torch.tensor(y[:-i])
    
    forecast = pipeline.predict(
        context= y,
        prediction_length=horizon,
        num_samples=20
    )
    
    low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)

    # store the results
    day_1.append(median[0])
    day_2.append(median[1])
    day_3.append(median[2])
    day_4.append(median[3])
    day_5.append(median[4])


In [ ]:
# Clean up the results
day_1 = day_1[-200:]
day_2 = day_2[3:-1]
day_3 = day_3[2:-2]
day_4 = day_4[1:-3]
day_5 = day_5[0:-4]
print(len(day_1))
print(len(day_2))
print(len(day_3))
print(len(day_4))
print(len(day_5))

In [ ]:
# Visualise time series with predictions with one day ahead
plt.figure(figsize=(8, 4))
plt.plot(relative_aquifer['date'][-1000:], y[-1000:], color="royalblue", label="historical data")
plt.plot(relative_aquifer['date'][-day_len:], day_1, color="tomato", label="median forecast")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Visualise time series with predictions with one day ahead
plt.figure(figsize=(8, 4))
plt.plot(relative_aquifer['date'][-1000:], y[-1000:], color="royalblue", label="historical data")
plt.plot(relative_aquifer['date'][-day_len:], day_2, color="tomato", label="median forecast")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Visualise time series with predictions with one day ahead
plt.figure(figsize=(8, 4))
plt.plot(relative_aquifer['date'][-1000:], y[-1000:], color="royalblue", label="historical data")
plt.plot(relative_aquifer['date'][-day_len:], day_3, color="tomato", label="median forecast")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Visualise time series with predictions with one day ahead
plt.figure(figsize=(8, 4))
plt.plot(relative_aquifer['date'][-1000:], y[-1000:], color="royalblue", label="historical data")
plt.plot(relative_aquifer['date'][-day_len:], day_4, color="tomato", label="median forecast")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Visualise time series with predictions with one day ahead
plt.figure(figsize=(8, 4))
plt.plot(relative_aquifer['date'][-1000:], y[-1000:], color="royalblue", label="historical data")
plt.plot(relative_aquifer['date'][-day_len:], day_5, color="tomato", label="median forecast")
plt.legend()
plt.grid()
plt.show()

Evaluation

In [ ]:
r2_1_day = r2_score(relative_aquifer['altitude_diff'][-day_len:], day_1)
r2_2_day = r2_score(relative_aquifer['altitude_diff'][-day_len:], day_2)
r2_3_day = r2_score(relative_aquifer['altitude_diff'][-day_len:], day_3)
r2_4_day = r2_score(relative_aquifer['altitude_diff'][-day_len:], day_4)
r2_5_day = r2_score(relative_aquifer['altitude_diff'][-day_len:], day_5)

print(f"1 day ahead: {r2_1_day}")
print(f"2 days ahead: {r2_2_day}")
print(f"3 days ahead: {r2_3_day}")
print(f"4 days ahead: {r2_4_day}")
print(f"5 days ahead: {r2_5_day}")

Evaluation for cumulative altitude (one day ahead)

In [ ]:
# Change the predicted relative differences to the absolute altitudes

# Get the last day_len + 1 days without the last one
altitudes = relative_aquifer['altitude'][-(day_len+1):-1]

# Sum original altitudes and relative differences
altitudes = altitudes + day_1

In [ ]:
# Calculate the r2 score
r2_score(relative_aquifer['altitude'][-day_len:], altitudes)

Testing multiple prediction horizons (with scaling)

In [ ]:
relative_aquifer = aquifer_by_stations[85065]

In [ ]:
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-large",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

horizon = 5
history = 100

day_1 = []
day_2 = []
day_3 = []
day_4 = []
day_5 = []

day_len = 200

# Iterate from day_len days before the end, to the last day
for i in range(day_len + 4, 0, -1):
    y = relative_aquifer['altitude'].values
    y_scaled = standard_scaling(y)
    y = torch.tensor(y_scaled[:-i])
    
    forecast = pipeline.predict(
        context= y,
        prediction_length=horizon,
        num_samples=20
    )
    
    low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)

    # Unscale the predictions
    median = standard_unscaling(relative_aquifer['altitude'], median)

    #store the results
    day_1.append(median[0])
    day_2.append(median[1])
    day_3.append(median[2])
    day_4.append(median[3])
    day_5.append(median[4])


In [ ]:
# Clean up the results
day_1 = day_1[-200:]
day_2 = day_2[3:-1]
day_3 = day_3[2:-2]
day_4 = day_4[1:-3]
day_5 = day_5[0:-4]
print(len(day_1))
print(len(day_2))
print(len(day_3))
print(len(day_4))
print(len(day_5))

In [ ]:
# Visualise time series with predictions with one day ahead
plt.figure(figsize=(8, 4))
plt.plot(relative_aquifer['date'][-1000:], relative_aquifer['altitude'][-1000:], color="royalblue", label="historical data")
plt.plot(relative_aquifer['date'][-day_len:], day_1, color="tomato", label="median forecast")
plt.legend()
plt.grid()
plt.show()

Evaluation

In [ ]:
r2_1_day = r2_score(relative_aquifer['altitude'][-day_len:], day_1)
r2_2_day = r2_score(relative_aquifer['altitude'][-day_len:], day_2)
r2_3_day = r2_score(relative_aquifer['altitude'][-day_len:], day_3)
r2_4_day = r2_score(relative_aquifer['altitude'][-day_len:], day_4)
r2_5_day = r2_score(relative_aquifer['altitude'][-day_len:], day_5)

print(f"1 day ahead: {r2_1_day}")
print(f"2 days ahead: {r2_2_day}")
print(f"3 days ahead: {r2_3_day}")
print(f"4 days ahead: {r2_4_day}")
print(f"5 days ahead: {r2_5_day}")

Averaging results for multiple aquifer stations

In [ ]:
aquifers_list = [85065, 85064]

In [ ]:
# Remove the last 5 days
# This is done to enable direct comparison to the randomforest,
# there the 5 days are removed because of the weather forecast generation
for aquifer in aquifers_list:
    aquifer_by_stations[aquifer] = aquifer_by_stations[aquifer][:-5]

In [ ]:
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-large",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

horizon = 5
day_len = 365

# List for r2 results for different prediction horizons
r2_scores = [[] for _ in range(5)]

# Create a dictionary for the predictions from all of the different aquifers
predictions_by_stations = {key: [] for key in aquifers_list}

for aquifer in aquifers_list:
    # List for storing the predictions
    predictions = [[] for _ in range(5)]


    # Iterate from day_len days before the end, to the last day
    for i in range(day_len + (horizon-1), 0, -1):
        
        y = aquifer_by_stations[aquifer]['altitude_diff'].values
        y = torch.tensor(y[:-i])

        forecast = pipeline.predict(
            context= y,
            prediction_length=horizon
        )

        median = np.quantile(forecast[0].numpy(), 0.5, axis=0)

        # Store the results for every prediction horizon separately
        for i in range(5):
            predictions[i].append(median[i])
    
    # Clean up the results
    predictions[0] = predictions[0][-day_len:]
    predictions[1] = predictions[1][3:-1]
    predictions[2] = predictions[2][2:-2]
    predictions[3] = predictions[3][1:-3]
    predictions[4] = predictions[4][0:-4]

    # Add the predictios to the dictionary
    predictions_by_stations[aquifer] = predictions

    # Calculate the r2 scores and store them in a list
    for i in range(5):
        r2_scores[i].append(r2_score(aquifer_by_stations[aquifer]['altitude_diff'][-day_len:], predictions[i]))

In [ ]:
# Visualise time series with predictions with one day ahead
plt.figure(figsize=(8, 4))
plt.plot(aquifer_by_stations[aquifer]['date'][-50:], aquifer_by_stations[aquifer]['altitude_diff'][-50:], color="royalblue", label="historical data")
#plt.plot(aquifer_by_stations[aquifer]['date'][-50:], predictions[0][-50:], color="tomato", label="median forecast")
#plt.plot(aquifer_by_stations[aquifer]['date'][-50:], predictions[1][-50:], color="green", label="median forecast")
#plt.plot(aquifer_by_stations[aquifer]['date'][-50:], predictions[2][-50:], color="orange", label="median forecast")
plt.plot(aquifer_by_stations[aquifer]['date'][-50:], predictions_by_stations[85065][0][-50:], color="orange", label="median forecast")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Calculate the average r2 score
r2_average =  []
std_dev = []

for i in range(5):
    r2_average.append(np.mean(r2_scores[i]))
    std_dev.append(np.std(r2_scores[i]))

In [ ]:
r2_average

In [ ]:
# Save the average r2_scores
with open('../reports/chronos-large/chronos-large-ground-water-r2.txt', 'w') as file:
    for item in r2_average:
        file.write(f"{item}\n")

In [ ]:
# Save the standard deviations
with open('../reports/chronos-large/chronos-large-ground-water-std-dev.txt', 'w') as file:
    for item in std_dev:
        file.write(f"{item}\n")

In [ ]:
# Transpose the r2_scores list
r2_scores_transposed = [list(x) for x in zip(*r2_scores)]
# Pair up the stations with their r2_scores and store them in a dictionary
scores = dict(zip(aquifers_list, r2_scores_transposed))
scores

In [ ]:
# Sort them by the value in r2_scores[0]
scores_sorted = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1][0])}
scores_sorted

In [ ]:
# Save the r2_scores
joblib.dump(scores_sorted, '../reports/chronos-large/chronos-large-ground-water-r2-stations.joblib')

In [ ]:
# Save the dictionary with predictions
joblib.dump(predictions_by_stations, '../reports/chronos-large/chronos-large-ground-water-predictions.joblib')